In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import ParameterGrid

In [44]:
df = pd.read_csv('final_combined_dataset.csv')


In [45]:
df = df.drop(columns=['nearby_cases_weighted','city','week','population','tempe_min','humidity_max','humidity_min','temp_max','vim','vim_monthly','precipitation_max_ordinary_kriging','precipitation_avg_regression_kriging','precipitation_max_regression_kriging','long','lat','cases_per_100k','nearby_cases_weighted'])

In [46]:
#geocode_list = [3300407, 3300456, 3302007, 3302205, 3303807]
geocode_list = [3303807]

df = df[df['geocode'].isin(geocode_list)]

# List of features to lag
features_to_lag = [
     "humidity_avg",
    "temp_avg","precipitation_avg_ordinary_kriging",
]

# Create lags for all features in features_to_lag for 1 to 12 weeks
lags = range(6, 7)

def create_lags(group, features, lags):
    for feature in features:
        for lag in lags:
            group[f"{feature}_lag_{lag}"] = group[feature].shift(lag)
    return group

# Group by geocode and apply lag creation
df = df.groupby('geocode', group_keys=False).apply(create_lags, features=features_to_lag, lags=lags)
df

C:\Users\user\AppData\Local\Temp\ipykernel_16660\1038510214.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('geocode', group_keys=False).apply(create_lags, features=features_to_lag, lags=lags)


,date,cases,humidity_avg,temp_avg,geocode,precipitation_avg_ordinary_kriging,humidity_avg_lag_6,temp_avg_lag_6,precipitation_avg_ordinary_kriging_lag_6
29274,2012-01-01,1,55.514486,25.048951,3303807,5.6804,NaN,NaN,NaN
29275,2012-01-08,0,62.357393,23.737513,3303807,3.9412,NaN,NaN,NaN
29276,2012-01-15,2,65.236264,24.413187,3303807,1.5898,NaN,NaN,NaN
29277,2012-01-22,3,60.362637,24.879121,3303807,1.1766,NaN,NaN,NaN
29278,2012-01-29,2,50.885924,25.989992,3303807,2.5209,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
29843,2022-11-27,0,79.717112,23.775511,3303807,4.9788,74.284890,24.017007,0.8428
29844,2022-12-04,3,77.826146,25.071428,3303807,1.8253,69.711634,25.122449,2.3012
29845,2022-12-11,3,77.462712,23.904762,3303807,3.2077,78.428065,20.429847,3.9107
29846,2022-12-18,8,82.448150,22.517858,3303807,2.4548,77.539105,21.608164,3.5073


In [47]:
df = df.drop(columns=['precipitation_avg_ordinary_kriging','geocode','temp_avg','humidity_avg'])

df.tail(20)

,date,cases,humidity_avg_lag_6,temp_avg_lag_6,precipitation_avg_ordinary_kriging_lag_6
29828,2022-08-14,1,67.596171,18.836735,0.0433
29829,2022-08-21,3,67.703375,20.614116,0.4046
29830,2022-08-28,2,66.905040,22.030612,0.5645
29831,2022-09-04,2,68.034333,19.767857,0.8046
29832,2022-09-11,5,74.856580,18.361224,0.1328
29833,2022-09-18,2,74.431393,19.193877,1.2327
29834,2022-09-25,1,71.758797,17.921429,0.5131
29835,2022-10-02,3,64.482619,20.408164,0.0869
29836,2022-10-09,3,74.855336,16.163265,0.6343
29837,2022-10-16,5,61.107726,22.482993,0.3189


In [48]:
df_cleaned = df.dropna()
df=df_cleaned
df

,date,cases,humidity_avg_lag_6,temp_avg_lag_6,precipitation_avg_ordinary_kriging_lag_6
29280,2012-02-12,3,55.514486,25.048951,5.6804
29281,2012-02-19,3,62.357393,23.737513,3.9412
29282,2012-02-26,2,65.236264,24.413187,1.5898
29283,2012-03-04,2,60.362637,24.879121,1.1766
29284,2012-03-11,4,50.885924,25.989992,2.5209
...,...,...,...,...,...
29843,2022-11-27,0,74.284890,24.017007,0.8428
29844,2022-12-04,3,69.711634,25.122449,2.3012
29845,2022-12-11,3,78.428065,20.429847,3.9107
29846,2022-12-18,8,77.539105,21.608164,3.5073


In [49]:
df.isnull().sum()

date                                        0
cases                                       0
humidity_avg_lag_6                          0
temp_avg_lag_6                              0
precipitation_avg_ordinary_kriging_lag_6    0
dtype: int64

In [50]:
df['date'] = pd.to_datetime(df['date'])

train_df = df[df['date'].dt.year <= 2020]
test_df = df[df['date'].dt.year >= 2021]

print("Training set shape:", train_df.shape)
print("Test set shape:", test_df.shape)

Training set shape: (464, 5)
Test set shape: (104, 5)


In [51]:
train_df['date'] = pd.to_datetime(train_df['date'])
train_df.set_index('date', inplace=True)

test_df['date'] = pd.to_datetime(test_df['date'])
test_df.set_index('date', inplace=True)


C:\Users\user\AppData\Local\Temp\ipykernel_16660\3654931774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['date'] = pd.to_datetime(train_df['date'])
C:\Users\user\AppData\Local\Temp\ipykernel_16660\3654931774.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['date'] = pd.to_datetime(test_df['date'])


In [52]:
X_train = train_df.drop(columns=['cases'])
y_train = train_df[['cases']]

X_test = test_df.drop(columns=['cases'])
y_test = test_df[['cases']]

In [53]:
assert X_train.index.equals(y_train.index), "Indices of X_train and y_train are not aligned."
assert X_test.index.equals(y_test.index), "Indices of X_test and y_test are not aligned."


In [54]:
y_train.index

DatetimeIndex(['2012-02-12', '2012-02-19', '2012-02-26', '2012-03-04',
               '2012-03-11', '2012-03-18', '2012-03-25', '2012-04-01',
               '2012-04-08', '2012-04-15',
               ...
               '2020-10-25', '2020-11-01', '2020-11-08', '2020-11-15',
               '2020-11-22', '2020-11-29', '2020-12-06', '2020-12-13',
               '2020-12-20', '2020-12-27'],
              dtype='datetime64[ns]', name='date', length=464, freq=None)

In [55]:
y_test

,cases
date,
2021-01-03,2
2021-01-10,2
2021-01-17,3
2021-01-24,2
2021-01-31,2
...,...
2022-11-27,0
2022-12-04,3
2022-12-11,3


In [56]:
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()
columns_to_scale = ['cases']
numeric_cols = [
    col for col in df.select_dtypes(include=['float64', 'int64', 'int32']).columns 
    if col not in ['cases'] + columns_to_scale
]
# Scale the selected numeric features
X_train[numeric_cols] = feature_scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = feature_scaler.transform(X_test[numeric_cols])

In [57]:
X_train

,humidity_avg_lag_6,temp_avg_lag_6,precipitation_avg_ordinary_kriging_lag_6
date,,,
2012-02-12,0.297788,0.769386,0.501669
2012-02-19,0.417567,0.707616,0.348070
2012-02-26,0.467959,0.739441,0.140404
2012-03-04,0.382651,0.761387,0.103912
2012-03-11,0.216770,0.813711,0.222635
...,...,...,...
2020-11-29,0.419770,0.746894,0.082531
2020-12-06,0.451292,0.782609,0.336289
2020-12-13,0.484064,0.609546,0.081727


In [58]:
y_train

,cases
date,
2012-02-12,3
2012-02-19,3
2012-02-26,2
2012-03-04,2
2012-03-11,4
...,...
2020-11-29,2
2020-12-06,0
2020-12-13,0


In [59]:
X_test

,humidity_avg_lag_6,temp_avg_lag_6,precipitation_avg_ordinary_kriging_lag_6
date,,,
2021-01-03,0.397625,0.816215,0.134885
2021-01-10,0.649444,0.673290,0.314219
2021-01-17,0.730143,0.684605,0.556187
2021-01-24,0.561227,0.778948,0.347019
2021-01-31,0.728821,0.708863,0.633304
...,...,...,...
2022-11-27,0.626347,0.720780,0.074433
2022-12-04,0.546296,0.772848,0.203232
2022-12-11,0.698869,0.551820,0.345377


In [60]:
y_test

,cases
date,
2021-01-03,2
2021-01-10,2
2021-01-17,3
2021-01-24,2
2021-01-31,2
...,...
2022-11-27,0
2022-12-04,3
2022-12-11,3


In [61]:
y_test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 104 entries, 2021-01-03 to 2022-12-25
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   cases   104 non-null    int64
dtypes: int64(1)
memory usage: 1.6 KB


In [ ]:
# Import required libraries

# X_train['date'] = pd.to_datetime(X_train['date'])
# X_train.set_index('date', inplace=True)

# X_test['date'] = pd.to_datetime(X_test['date'])
# X_test.set_index('date', inplace=True)


sarima_order = (1, 0, 0)  # (p, d, q)
seasonal_order = (2, 0, 0, 52)  # (P, D, Q, s)

# Fit SARIMAX model on training data
model = SARIMAX(y_train['cases'], exog=X_train, 
                order=sarima_order, 
                seasonal_order=seasonal_order, 
                enforce_stationarity=False, 
                enforce_invertibility=False)
fit_model = model.fit(disp=False)
print("model")
# Rolling forecast
rolling_predictions = []
history_y = list(y_train['cases'])
history_X = X_train.copy()
forecast_horizon = 1  # Forecast one step at a time

for i in range(len(y_test)):
    try:
        # Prepare the current exogenous variables
        exog_future = X_test.iloc[i:i + forecast_horizon]

        # Forecast the next step
        forecast = fit_model.forecast(steps=forecast_horizon, exog=exog_future)
        rolling_predictions.extend(forecast)

        # Append the actual value to history for refitting
        actual_value = y_test.iloc[i]['cases']
        history_y.append(actual_value)
        new_exog = exog_future.iloc[0]
        history_X = pd.concat([history_X, pd.DataFrame([new_exog], columns=history_X.columns)])
        
        # Update the model with the extended history
        fit_model = SARIMAX(history_y, exog=history_X, 
                            order=sarima_order, 
                            seasonal_order=seasonal_order, 
                            enforce_stationarity=False, 
                            enforce_invertibility=False).fit(disp=False)
        
    except Exception as e:
        print(f"Failed to forecast at iteration {i}: {e}")

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, rolling_predictions))
print(f"RMSE for rolling forecast with exogenous variables: {rmse:.2f}")

# Plot actual vs predicted values
plt.figure(figsize=(12, 6))
plt.plot(range(len(y_test)), y_test, label='Actual', color='blue')
plt.plot(range(len(y_test)), rolling_predictions, label='Predicted (Rolling)', color='red')
plt.title(f'Actual vs Predicted Cases (Rolling Forecast with Exogenous Variables) | RMSE: {rmse:.2f}')
plt.xlabel('Time Step')
plt.ylabel('Cases')
plt.legend()
plt.show()


c:\Program Files\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Program Files\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


model


c:\Program Files\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Program Files\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Program Files\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Program Files\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error


# X_train['date'] = pd.to_datetime(X_train['date'])
# X_train.set_index('date', inplace=True)

# X_test['date'] = pd.to_datetime(X_test['date'])
# X_test.set_index('date', inplace=True)

# Define SARIMAX parameters
sarima_order = (1, 0, 0)  # (p, d, q)
seasonal_order = (2, 0, 0, 52)  # (P, D, Q, s)

# Fit SARIMAX model on training data
model = SARIMAX(y_train['cases'], exog=X_train, 
                order=sarima_order, 
                seasonal_order=seasonal_order, 
                enforce_stationarity=False, 
                enforce_invertibility=False)
fit_model = model.fit(disp=False)

# Rolling forecast
rolling_predictions = []
history_y = list(y_train['cases'])
history_X = X_train.copy()
forecast_horizon = 1

for i in range(len(y_test)):
    try:
        # Prepare the current exogenous variables
        exog_future = X_test.iloc[i:i + forecast_horizon]

        # Forecast the next step
        forecast = fit_model.forecast(steps=forecast_horizon, exog=exog_future)
        rolling_predictions.extend(forecast)

        # Append the actual value to history for refitting
        actual_value = y_test.iloc[i]['cases']
        history_y.append(actual_value)
        new_exog = exog_future.iloc[0]
        history_X = pd.concat([history_X, pd.DataFrame([new_exog], columns=history_X.columns)])

        # Update the model with the extended history
        fit_model = SARIMAX(history_y, exog=history_X, 
                            order=sarima_order, 
                            seasonal_order=seasonal_order, 
                            enforce_stationarity=False, 
                            enforce_invertibility=False).fit(disp=False)

    except Exception as e:
        print(f"Failed to forecast at iteration {i}: {e}")

# Print results
print("Rolling Predictions:", rolling_predictions)

